In [1]:
from collections import namedtuple

# other packages
import pandas as pd
# pytorch
import torch
import torch.optim as optim
import tqdm
tqdm.tqdm.pandas()
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.vocabulary import Vocabulary
# model
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.training.learning_rate_schedulers import CosineWithRestarts, LearningRateScheduler
from allennlp.training.learning_rate_schedulers import LearningRateScheduler
# attention
from allennlp.modules.attention import AdditiveAttention
from allennlp.modules.seq2seq_encoders import StackedSelfAttentionEncoder
# encoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.training.trainer import Trainer
# preprocessing
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

/home/analekseev/anaconda3/envs/NLPHW3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
torch.cuda.is_available()

True

In [3]:


Config = namedtuple('Config', [
                    'lazy',
                    'max_vocab_size', 
                    'batch_size', 
                    'epochs', 
                    'max_seq_len', 
                    'IN_EMBEDDING_DIM', 
                    'HIDDEN_DIM', 
                    'OUT_EMBEDDING_DIM', 
                    'CUDA_DEVICE',
                    'n_samples'
])



# create config file

In [4]:
config = Config(False,
               10000,
               64,
               40,
               100,
               64,
               32,
               64,
               0 if torch.cuda.is_available() else -1, 
               100000)

# prepare data

In [5]:
raw_dataset = pd.read_csv('data/ru_train.csv')

raw_dataset['before'] = raw_dataset['before'].astype(str)
raw_dataset['after'] = raw_dataset['after'].astype(str)

d = raw_dataset['class'].value_counts().to_dict()

for i in d:
    if d[i] > config.n_samples:
        d[i] = config.n_samples

In [6]:
d

{'PLAIN': 100000,
 'PUNCT': 100000,
 'CARDINAL': 100000,
 'LETTERS': 100000,
 'DATE': 100000,
 'VERBATIM': 100000,
 'ORDINAL': 46738,
 'MEASURE': 40534,
 'TELEPHONE': 10088,
 'DECIMAL': 7297,
 'ELECTRONIC': 5832,
 'MONEY': 2690,
 'FRACTION': 2460,
 'DIGIT': 2012,
 'TIME': 1945}

In [7]:
rus = RandomUnderSampler(sampling_strategy=d, random_state=0)

raw_dataset_resampled, _ = rus.fit_resample(raw_dataset, raw_dataset['class'])

In [8]:
df = raw_dataset_resampled[['before', 'after']]

df_train, df_test = train_test_split(df, test_size=0.1)

df_train.to_csv('train_dataset.tsv', index=False, header=False, sep='\t')
df_test.to_csv('test_dataset.tsv', index=False, header=False, sep='\t')

# create reader

In [9]:
reader = Seq2SeqDatasetReader(
    source_tokenizer = CharacterTokenizer(),
    target_tokenizer = CharacterTokenizer(),
    source_token_indexers={'tokens': SingleIdTokenIndexer()},
    target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')},
    lazy=config.lazy
)

train_dataset = reader.read('train_dataset.tsv',)
validation_dataset = reader.read('test_dataset.tsv')

647636it [00:29, 22182.89it/s]
71960it [00:04, 14582.36it/s]


# prepare vocabulary

In [10]:
vocab = Vocabulary.from_instances(train_dataset,
                                  min_count={'tokens': 3, 'target_tokens': 3}
                                 )

in_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=config.IN_EMBEDDING_DIM
                        )

source_embedder = BasicTextFieldEmbedder({"tokens": in_embedding})

iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("source_tokens", "num_tokens")],
                         )

iterator.index_with(vocab)

100%|██████████| 647636/647636 [00:07<00:00, 92033.41it/s]


# prepare model

In [11]:
## get simple encoder
encoder = StackedSelfAttentionEncoder(input_dim=config.IN_EMBEDDING_DIM, 
                                      hidden_dim=config.HIDDEN_DIM, 
                                      projection_dim=64, 
                                      feedforward_hidden_dim=64, 
                                      num_layers=1, 
                                      num_attention_heads=8)

attention = AdditiveAttention(config.HIDDEN_DIM, config.HIDDEN_DIM)

In [12]:
max_decoding_steps = 100

model = SimpleSeq2Seq(vocab, 
                      source_embedder, 
                      encoder, 
                      max_decoding_steps,
                      target_embedding_dim=config.OUT_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      beam_size=8,
                      use_bleu=False,
                      attention=attention,
                      scheduled_sampling_ratio = 0.15)

In [13]:
if torch.cuda.is_available():
    model.cuda(config.CUDA_DEVICE)

In [14]:
optimizer = optim.Adam(model.parameters())
lrs = CosineWithRestarts(optimizer, 2)

# training

In [15]:
torch.cuda.is_available()

True

In [16]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  num_epochs=config.epochs,
                  cuda_device=config.CUDA_DEVICE, 
                  learning_rate_scheduler=lrs)

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


In [17]:
print(f'Will train for {config.epochs} epochs')
trainer.train()

  0%|          | 0/10120 [00:00<?, ?it/s]

Will train for 40 epochs


loss: 1.3713 ||: 100%|██████████| 10120/10120 [09:49<00:00, 17.16it/s]
  0%|          | 0/1125 [00:00<?, ?it/s]/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
loss: 0.6883 ||: 100%|██████████| 1125/1125 [01:46<00:00, 10.58it/s]
loss: 0.7354 ||: 100%|██████████| 10120/10120 [09:36<00:00, 17.56it/s]
loss: 0.4944 ||: 100%|██████████| 1125/1125 [01:53<00:00,  9.95it/s]
loss: 0.5733 ||: 100%|██████████| 10120/10120 [09:30<00:00, 17.72it/s]
loss: 0.3268 ||: 100%|██████████| 1125/1125 [01:57<00:00,  9.61it/s]
loss: 0.4552 ||: 100%|██████████| 10120/10120 [09:41<00:00, 17.40it/s]
loss: 0.2628 ||: 100%|██████████| 1125/1125 [01:53<00:00,  9.92it/s]
loss: 0.4132 ||: 100%|██████████| 10120/10120 [09:33<00:00, 17.64it/s]
loss: 0.2217 ||: 100%|██████████| 1125/1125 

KeyboardInterrupt: 

In [22]:
with open("Transformer-alt.th", 'wb') as f:
    torch.save(model.state_dict(), f)
    
tqdm.tqdm.pandas()

model.eval()

predictor = SimpleSeq2SeqPredictor(model, reader)


In [23]:
kaggle_test = pd.read_csv('data/ru_test_2.csv')
kaggle_test['before'] = kaggle_test['before'].astype(str)

small_kaggle = pd.DataFrame(kaggle_test['before'].unique(), columns=['before'])

In [24]:
small_kaggle['after'] = small_kaggle['before'].progress_apply(lambda x: ''.join(predictor.predict(x.lower())['predicted_tokens']))

100%|██████████| 175991/175991 [1:06:12<00:00, 44.31it/s]


In [25]:
small_kaggle = small_kaggle.set_index('before')

In [26]:
kaggle_test = kaggle_test.join(small_kaggle, on='before')
kaggle_test['id'] = kaggle_test['sentence_id'].astype(str)+'_'+kaggle_test['token_id'].astype(str)

In [27]:
kaggle_test[['id', 'after']].to_csv('sub-alt.csv', index=False)